In [1]:
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

In [2]:
# Cargar el dataset de MNIST fashion

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [4]:
# Instalar librería keras_tuner en Colab

if "google.colab" in sys.modules:
    %pip install -q -U keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


En caso de no usar Colab ver:

https://keras.io/keras_tuner/

In [5]:
import keras_tuner as kt

def build_model(hiper):
    # Definir el número de capas ocultas como un hiperparámetro ajustable
    n_hidden = hiper.Int("n_hidden", min_value=1, max_value=8)

    # Definir el número de neuronas por capa como un hiperparámetro ajustable
    n_neurons = hiper.Int("n_neurons", min_value=16, max_value=256)

    # Definir el learning rate como un hiperparámetro ajustable en escala logarítmica
    learning_rate = hiper.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")

    # Definir el optimizador como como una opción entre "sgd" y "adam"
    optimizer = hiper.Choice("optimizer", values=["sgd", "adam"])

    # Configurar el optimizador en función de la opción elegida
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Construir el modelo secuencial de Keras
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [6]:
# Esta celda tardar varias minutos en ejecutarse

# Crear un sintonizador de búsqueda aleatoria
random_search_tuner = kt.RandomSearch(
    build_model,  # Función que construye el modelo a optimizar
    objective="val_accuracy",  # Objetivo de optimización: precisión en datos de validación
    max_trials=5,  # Número máximo de combinaciones de hiperparámetros a probar
    overwrite=True,  # Sobrescribir los resultados anteriores si existe un experimento con el mismo nombre
    directory="my_fashion_mnist",  # Directorio donde se guardarán los resultados y checkpoints del sintonizador
    project_name="my_rnd_search",  # Nombre del proyecto de búsqueda aleatoria
    seed=42  # Semilla para reproducibilidad
)

# Ejecutar la búsqueda aleatoria con los datos de entrenamiento y validación
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

random_search_tuner.search(
    X_train, y_train,  # Datos de entrenamiento
    epochs=20,  # Número de épocas de entrenamiento por cada configuración de hiperparámetros
    validation_data=(X_valid, y_valid),  # Datos de validación para evaluar el rendimiento del modelo
    callbacks=[early_stopping] # Implementar early stopping (parada temprana)
)


Trial 5 Complete [00h 02m 07s]
val_accuracy: 0.8532000184059143

Best val_accuracy So Far: 0.8636000156402588
Total elapsed time: 00h 06m 04s


In [7]:
# Obtener los 3 mejores modelos según la búsqueda aleatoria
top3_models = random_search_tuner.get_best_models(num_models=3)

# Seleccionar el mejor modelo de los tres mejores modelos obtenidos
best_model = top3_models[0]

# Devolver el mejor modelo seleccionado
best_model

In [8]:
# Obtener los mejores hiperparámetros de los tres mejores modelos encontrados
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)

# Obtener los valores de los hiperparámetros del primer conjunto de mejores hiperparámetros
top3_params[0].values

{'n_hidden': 8,
 'n_neurons': 100,
 'learning_rate': 0.0012482904754698163,
 'optimizer': 'sgd'}

In [9]:
# Obtener el mejor ensayo (trial) según el oráculo de búsqueda aleatoria
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]

# Mostrar un resumen del mejor ensayo
best_trial.summary()

Trial 1 summary
Hyperparameters:
n_hidden: 8
n_neurons: 100
learning_rate: 0.0012482904754698163
optimizer: sgd
Score: 0.8636000156402588


In [10]:
# Obtener el último valor registrado de la métrica "val_accuracy" del mejor ensayo
best_trial.metrics.get_last_value("val_accuracy")

0.8636000156402588

In [11]:
# Entrenar el mejor modelo en el dataset de entrenamiento completo
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = best_model.fit(X_train_full, y_train_full, epochs=100, callbacks=[early_stopping])

Epoch 1/100
1871/1875 [============================>.] - ETA: 0s - loss: 0.3241 - accuracy: 0.8810

1875/1875 [==============================] - 9s 4ms/step - loss: 0.3242 - accuracy: 0.8810
Epoch 2/100
1871/1875 [============================>.] - ETA: 0s - loss: 0.3161 - accuracy: 0.8832

1875/1875 [==============================] - 8s 4ms/step - loss: 0.3162 - accuracy: 0.8832
Epoch 3/100
1872/1875 [============================>.] - ETA: 0s - loss: 0.3069 - accuracy: 0.8869

1875/1875 [==============================] - 7s 4ms/step - loss: 0.3071 - accuracy: 0.8869
Epoch 4/100
1871/1875 [============================>.] - ETA: 0s - loss: 0.2998 - accuracy: 0.8897

1875/1875 [==============================] - 8s 4ms/step - loss: 0.2998 - accuracy: 0.8897
Epoch 5/100
1869/1875 [============================>.] - ETA: 0s - loss: 0.2946 - accuracy: 0.8910

...

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0929 - accuracy: 0.9657
Epoch 97/100
1874/1875 [============================>.] - ETA: 0s - loss: 0.0935 - accuracy: 0.9645

1875/1875 [==============================] - 8s 4ms/step - loss: 0.0936 - accuracy: 0.9646
Epoch 98/100
1874/1875 [============================>.] - ETA: 0s - loss: 0.0935 - accuracy: 0.9644

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0935 - accuracy: 0.9644
Epoch 99/100
1865/1875 [============================>.] - ETA: 0s - loss: 0.0920 - accuracy: 0.9660

1875/1875 [==============================] - 8s 4ms/step - loss: 0.0920 - accuracy: 0.9660
Epoch 100/100
1873/1875 [============================>.] - ETA: 0s - loss: 0.0916 - accuracy: 0.9660

1875/1875 [==============================] - 8s 4ms/step - loss: 0.0916 - accuracy: 0.9660


In [12]:
# Evaluar el mejor modelo con los datos de prueba y obtener la pérdida y precisión
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

print(test_loss)
print(test_accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.6410 - accuracy: 0.8657
0.6410060524940491
0.8657000064849854
